### We are assuming that the state vector are in the following order:
$$ x = [E_{qp}\quad E_{dp}\quad \delta\quad \omega\quad V_F\quad V_A\quad V_E]^T $$

In [1]:
# from rushisland_Hopf import *
from Hopf_ESDC1A import *

In [ ]:
who

In [2]:
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import optimize
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(precision=25)

In [3]:
sys_fun(x)
sol = optimize.root(sys_fun, x, method='hybr')
print(sol.fun)
print(sol.message)
print(sol.success)

[ 0.0000000000000000e+00  0.0000000000000000e+00 -1.9001570304084425e-30
  0.0000000000000000e+00  0.0000000000000000e+00  3.2959746043559335e-14
 -5.6934514083341356e-16  4.1659400548786359e-17 -5.6166426203633553e-16
 -1.1464915525482156e-30  2.1101886901879908e-17  0.0000000000000000e+00
  2.1394922870380622e-14 -2.8467257041670678e-16]
The solution converged.
True


In [4]:
x = sol.x

In [ ]:
abs(f_Vdq(x))

### Now we calculate Jacobian matrix $J$ and  the tranformation matrix $Q$

In [5]:
J = np.array([nd.Jacobian(f)(x).ravel() for f in all_fun])
# J = Jacobian(x)
lam, v = np.linalg.eig(J) # Here we want to calculate the left eigenvecs, so use J_org.T; because use left eigvec makes it easier to calc transformation matrix

lam = lam[[0,1,2,3,4,7,5,6,8,13,9,10,11,12]]
v = v[:,[0,1,2,3,4,7,5,6,8,13,9,10,11,12]]
v[:,[2,8]] = -v[:,[2,8]]

print(lam)

[-2.0412726082487723e+02+0.j
 -1.6662796466500484e+02+0.j
 -1.6527221505208093e+01+0.j
 -8.3185371194736391e-01+7.974543549087917j
 -8.3185371194736391e-01-7.974543549087917j
 -6.2448747415375507e+00+0.j
  5.5178084323870280e-14+5.5493916053923815j
  5.5178084323870280e-14-5.5493916053923815j
 -5.0444872495224224e+00+0.j
 -8.6823210703323028e-01+0.j
 -2.1827446947084859e+00+1.5091549543605527j
 -2.1827446947084859e+00-1.5091549543605527j
 -7.7387064005130446e-01+0.8366497990150229j
 -7.7387064005130446e-01-0.8366497990150229j]


In [6]:
n_c = 2
n_s = 12
n = x.shape[0]

In [ ]:
""" For Fred's Formula """
idx = 6
Q = np.c_[-v[:,idx].imag,v[:,idx].real]
i = 0
while i < len(lam):
    if i==idx or i==idx+1:
        i += 1
    else:
        Q=np.c_[Q,v[:,i]]
        i += 1
            
Q = Q/(abs(np.linalg.det(Q))**(1/n))
P = np.linalg.inv(Q)
J_cs = np.linalg.multi_dot([P,J,Q])
display(pd.DataFrame(J_cs).applymap(lambda x: '{:,.8f}'.format(x)))

In [ ]:
""" Weijun's Formula """
idx = 6
Q = np.c_[-v[:,idx].imag,v[:,idx].real]
i=0
while i < len(lam):
    if i==idx or i==idx+1:
        i += 1
    else:
        if lam[i].imag == 0:
            Q=np.c_[Q,v[:,i]]
            i += 1
        else:
            Q=np.c_[Q,v[:,i].imag,v[:,i].real]
            i += 2
            
Q = Q/(abs(np.linalg.det(Q))**(1/n))
P = np.linalg.inv(Q)
J_cs = np.linalg.multi_dot([P,J,Q])
display(pd.DataFrame(J_cs).applymap(lambda x: '{:,.8f}'.format(x)))

In [8]:
Z2 = np.zeros((n,n*(n+1)//2))
for i in range(n):
    hes = nd.Hessian(all_fun[i])(x) # The original Hessian of each f in all_fun
    #hes = Hessian(x)[i]
    hes[np.triu_indices(n,1)] *= 2  # double each element above the main diagonal
    Z2[i] = hes[np.triu_indices(n)] # Keep upper triangular part
Z2 = Z2/2 #divide all elements by 2, which corresponds to *2 in above line

In [9]:
Z3 = np.zeros((n,(n*(n+1)*(n+2)//6)))
Z3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
for i in range(n):
    t = Trissian(all_fun[i], x)
    #t = Trissian(x)[i]
    Z3[i] = [t[j] for j in Z3_idx]

Z3_Gain = []
for i in Z3_idx:
    val = 1
    for j in range(n):
        val *= math.factorial(i.count(j))
    Z3_Gain.append(val)
Z3_Gain = np.diag(1/np.array(Z3_Gain))    
Z3 = Z3.dot(Z3_Gain)
# np.savetxt("Fxxx.csv", Z3, delimiter=",")

# Z3 = np.genfromtxt('Fxxx_goodcase1.csv', delimiter=',',dtype=float)
# display(pd.DataFrame(Z3))

In [10]:
W2 = np.linalg.multi_dot([np.linalg.inv(Q), Z2, T2_mat(n), np.kron(Q,Q), S2_mat(n)])
# display(pd.DataFrame(W2)) # W2 is of dim: n x n(n+1)/2 = 14 x 105


W3 = np.linalg.multi_dot([np.linalg.inv(Q), Z3, T3_mat(n), np.kron(np.eye(n),T2_mat(n)), 
                      np.kron(np.kron(Q,Q), Q), 
                      np.kron(np.eye(n),S2_mat(n)), S3_mat(n)])

In [11]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,(0.14338531451499853+0j),(0.028273738442424416+0j),(-0.005354944834184397+0j),(1.7371661375131883e-05+0j),(-0.06926949805868893+0j),(0.055579397847469146-0.04519581392868273j),(0.055579397847469146+0.04519581392868273j),(0.20102187345928596+0j),(0.19209771072391943-0j),(0.017385441153121572+0j),(-0.23046122651934536-0.18589976860229046j),(-0.23046122651934536+0.18589976860229046j),(-0.272023527217051+0.1948634496368631j),(-0.272023527217051-0.1948634496368631j)
1,(0.12219407264383698+0j),(0.06821430224930901+0j),(2.4191184672266178e-06+0j),(-5.536475012788333e-06+0j),(-0.002042577908227252+0j),(0.02498477186717995-0.6855444009779165j),(0.02498477186717995+0.6855444009779165j),(0.7139516519594216+0j),(-0.21023031043216453+0j),(-0.013501064117045113+0j),(0.16967742775709518+0.08019700269370247j),(0.16967742775709518-0.08019700269370247j),(0.15192455657463344-0.16117936020923157j),(0.15192455657463344+0.16117936020923157j)
2,(0.3036407378209805+0j),(0.789997437763999+0j),(4.999956923447544e-06+0j),(-1.199638440431701e-06+0j),(0.00785478721760406-0j),(1.4765378820520652-1.0559720731819202j),(1.4765378820520652+1.0559720731819202j),(0.24314791873351518+0j),(-0.22919291565822697+0j),(-0.01702619751099167+0j),(0.2578632894788623+0.21588672609808499j),(0.2578632894788623-0.21588672609808499j),(0.2560243227671333-0.21235541733535485j),(0.2560243227671333+0.21235541733535485j)
3,(-0.011628934834479458+0j),(0.004469657981686247+0j),(-2.707298557247015e-06+0j),(5.302334773398566e-07+0j),(-0.00034435243133044117+0j),(0.019079048323826+0.03356346956862j),(0.019079048323826-0.03356346956862j),(-0.004027756151064768+0j),(0.003066811614903047-0j),(3.921230664866868e-05+0j),(-0.002357233913203843-0.0002176972909215046j),(-0.002357233913203843+0.0002176972909215046j),(-5.42787039129898e-05+0.0010041040820627715j),(-5.42787039129898e-05-0.0010041040820627715j)
4,(1.335226363237669+0j),(0.34870681490177496+0j),(-0.04152447399826565+0j),(0.00011712401815712528+0j),(-0.5640370808308308+0j),(0.07307223629144477-0.9437921883444603j),(0.07307223629144477+0.9437921883444603j),(1.5706324438195345+0j),(2.0241033957176087-0j),(-0.09631958168568786+0j),(-1.4645640371584352-3.013423977052948j),(-1.4645640371584352+3.013423977052948j),(-2.012478254128878-1.0156204901027845j),(-2.012478254128878+1.0156204901027845j)
5,(-0.02626643344234399+0j),(0.00615560517739768+0j),(-1.0207916944157747+0j),(0.0019126968030696698+0j),(-0.0774385181935493+0j),(0.002121959677072902+0.03610645339750156j),(0.002121959677072902-0.03610645339750156j),(0.022270439890254824+0j),(0.016292626766252396-0j),(-1.4467425443119498e-05+0j),(-0.002965987385640862+0.005073655274699168j),(-0.002965987385640862-0.005073655274699168j),(0.001025133311676657-0.0010012149084013585j),(0.001025133311676657+0.0010012149084013585j)
6,0j,(5.4613956552019935+0j),(5.8071067487789+0j),(-0.013349010345520518+0j),(5.868119301878108-0j),(5.224285518698671+0j),(5.224285518698671-0j),(-5.197165541067486+0j),(-5.021185606556923+0j),(-0.03861643187037771+0j),(3.8791362064829342+0j),(3.8791362064829342-0j),(-0.35156785979174504-1.6350891373014038j),(-0.35156785979174504+1.6350891373014038j)
7,(-0.034110735280719534+0j),(-0.0003919106627788989+0j),(2.8999050437946674e-06+0j),(-0.0024096369889976864+0j),(0.0011397422582152279-0j),(-0.06664589088844401-0.0962478226903493j),(-0.06664589088844401+0.0962478226903493j),(0.03543010590660975+0j),(-0.07790968008868314+0j),(-0.050854203560786244+0j),(-0.06697814019086216+0.19048770481479496j),(-0.06697814019086216-0.19048770481479496j),(0.4817975052181157-0.41940042802129196j),(0.4817975052181157+0.41940042802129196j)
8,(0.15193642630261875+0j),(-0.010411591729829719+0j),(1.3310752847068612e-05+0j),(1.3809014020624102e-06+0j),(0.004487190163674743-0j),(-0.05968829887783499+0.4794909724715261j),(-0.05968829887783499-0.4794909724715261j),(-0.8850773405514343+0j),(0.4321021747130741-0j),(0.023427197794882837+0j),(-0.06032151186661898-0.1282245

In [12]:
J_c = J_cs[0:n_c,0:n_c]
J_s = J_cs[n_c:,n_c:]
V2_uu = np.zeros((n_s,n_c*(n_c+1)//2))

W2_idx = [(i,j) for i in range(n) for j in range(i,n)]
V2_uu_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
V2_uu_idx = [W2_idx.index(i) for i in V2_uu_idx]
V2_uu = np.array([w[V2_uu_idx] for w in W2[n_c:n]])
# display(pd.DataFrame(V2_uu))

J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
#display(pd.DataFrame(J_cbar))

C2 = - np.linalg.multi_dot([T2_mat(n_c),J_cbar,S2_mat(n_c)])
#display(pd.DataFrame(C2))

H_c2 = sp.linalg.solve_sylvester(J_s, C2, -V2_uu)
# display(pd.DataFrame(H_c2))

In [13]:
Uu2_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
Uu2_idx = [W2_idx.index(i) for i in Uu2_idx]
Uu2 = np.array([w[Uu2_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uu2))
Uuv_idx = [(i,j) for i in range(n_c) for j in range(n_c,n)]
Uuv_idx = [W2_idx.index(i) for i in Uuv_idx]
Uuv = np.array([w[Uuv_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uuv))

In [14]:
W3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
Uu3_idx = [(i,j,k) for i in range(n_c) for j in range(i,n_c) for k in range(j,n_c)]
Uu3_idx = [W3_idx.index(i) for i in Uu3_idx]
Uu3 = np.array([w[Uu3_idx] for w in W3[0:n_c]])
#display(pd.DataFrame(Uu3))
Uuc3 = np.linalg.multi_dot([Uuv, np.kron(np.eye(n_c), H_c2), S3_mat(n_c)]) + Uu3

In [15]:
""" Guckenheimer's formula """
(Uuc3[0,0]*6 + 2*Uuc3[0,2] + 6*Uuc3[1,3] + 2*Uuc3[1,1])/16 - \
(Uu2[0,1]*(2*Uu2[0,0] + 2*Uu2[0,2]) - Uu2[1,1]*(2*Uu2[1,0] + 2*Uu2[1,2]) \
 - 2*Uu2[0,0]*2*Uu2[1,0] + 2*Uu2[0,2]*2*Uu2[1,2])/16/J_cs[0,1].real

(-0.22114233036131895-1.1784088885431932e-15j)

In [16]:
""" Fred's Formula """
Z2_idx = [(i,j) for i in range(n) for j in range(i,n)]

omega = J_cs[0,1].real
print(omega)
ac = (W3[0][Z3_idx.index((0,0,0))]*6 + W3[0][Z3_idx.index((0,1,1))]*2 + \
      W3[1][Z3_idx.index((1,1,1))]*6 + W3[1][Z3_idx.index((0,0,1))]*2)/16 - \
     (W2[0][Z2_idx.index((0,1))]*(W2[0][Z2_idx.index((0,0))]*2 + W2[0][Z2_idx.index((1,1))]*2) - 
      W2[1][Z2_idx.index((0,1))]*(W2[1][Z2_idx.index((0,0))]*2 + W2[1][Z2_idx.index((1,1))]*2) - \
      4*W2[0][Z2_idx.index((0,0))]*W2[1][Z2_idx.index((0,0))] + \
      4*W2[0][Z2_idx.index((1,1))]*W2[1][Z2_idx.index((1,1))])/16/omega
print("ac=",ac)
print("-----------------")
as_sum = 0
lamb = np.diag(J_cs[2:,2:])
for j in range(2,n):
    lj= lamb[j-2]
    print("lambda_j={0}:".format(lj))
    
    H21 = (                    -1/lj*(W2[j][Z2_idx.index((0,0))] + W2[j][Z2_idx.index((1,1))]) - \
               lj/(lj**2+4*omega**2)*(W2[j][Z2_idx.index((0,0))] - W2[j][Z2_idx.index((1,1))]) + \
          2*omega/(lj**2+4*omega**2)* W2[j][Z2_idx.index((0,1))]                                   )/2
    print("j={0}: H21={1}".format(j,H21))
    
    H22 =      -lj/(lj**2+4*omega**2)* W2[j][Z2_idx.index((0,1))] - \
           2*omega/(lj**2+4*omega**2)*(W2[j][Z2_idx.index((0,0))] - W2[j][Z2_idx.index((1,1))])
    print("j={0}: H22={1}".format(j,H22))
    
    H23 =  (                   -1/lj*(W2[j][Z2_idx.index((0,0))] + W2[j][Z2_idx.index((1,1))]) + \
               lj/(lj**2+4*omega**2)*(W2[j][Z2_idx.index((0,0))] - W2[j][Z2_idx.index((1,1))]) - \
          2*omega/(lj**2+4*omega**2)* W2[j][Z2_idx.index((0,1))]                                   )/2
    print("j={0}: H23={1}".format(j,H23))
    
    as_j =(W2[0][Z2_idx.index((0,j))]*(6*H21 + 2*H23) + 2*W2[0][Z2_idx.index((1,j))]*H22 +\
           W2[1][Z2_idx.index((1,j))]*(2*H21 + 6*H23) + 2*W2[1][Z2_idx.index((0,j))]*H22 ) / 16
    print("j={0}: as_{1}={2}".format(j,j,as_j))
    print("-----------------")
    as_sum += as_j

a = ac + as_sum
print("a=",a)

5.549391605392371
ac= (0.11171181417034694-1.371026068653606e-16j)
-----------------
lambda_j=(-204.12726082487723+7.939751949039377e-16j):
j=2: H21=(-0.018942027497605745+1.9797316059415207e-20j)
j=2: H22=(-0.08130823552469087+2.0686940199144298e-20j)
j=2: H23=(-0.08434319023064381-4.464245362478087e-20j)
j=2: as_2=(0.00015360193019796359+7.678510502514369e-20j)
-----------------
lambda_j=(-166.62796466500464+4.899203699875274e-16j):
j=3: H21=(-0.003531809590534967-7.148757690757299e-21j)
j=3: H22=(-0.014460408809549897-1.6571973292171863e-20j)
j=3: H23=(-0.013539246425480992-1.485895046318145e-20j)
j=3: as_3=(1.1690556641926563e-05+9.587246043773252e-21j)
-----------------
lambda_j=(-16.52722150520805+8.061826882539232e-16j):
j=4: H21=(0.3573461832836906-6.728682368641052e-17j)
j=4: H22=(0.790339392679333-1.1382583702159198e-16j)
j=4: H23=(0.5704665360235881-6.050284891716848e-17j)
j=4: as_4=(-0.008208181086933969+1.924211795683706e-18j)
-----------------
lambda_j=(-0.831853711947357

In [17]:
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2

In [18]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uuc3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(Ny3))
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(C3))

In [19]:
Lambda3 = np.kron(J_c,np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c),C3.T)
np.linalg.matrix_rank(Lambda3)

8

In [20]:
lam, _ = np.linalg.eig(Lambda3)
lam

array([-1.0658141036401503e-13+2.2197566421569505e+01j,
       -1.1762812945903534e-13-2.2197566421569515e+01j,
       -1.0824680508537336e-13+1.1098783210784745e+01j,
       -1.1455589743975170e-13-1.1098783210784752e+01j,
       -1.0658141036401503e-13+1.1098783210784749e+01j,
       -1.1246559239452836e-13-1.1098783210784763e+01j,
       -1.1108780519794997e-13+4.5582179018407759e-16j,
       -1.0931818420443435e-13-6.6121583455444592e-16j])

In [21]:
L3 = np.zeros((8,2))
L3[[0,2,5,7],[0,0,0,0]] = 1
L3[[1,3],[1,1]] = -1
L3[[4,6],[1,1]] = 1
L3
# np.linalg.matrix_rank(L3)
# print(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))))
# assert(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))) == Lambda3.shape[0])

array([[ 1.,  0.],
       [ 0., -1.],
       [ 1.,  0.],
       [ 0., -1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [22]:
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
L_inv = np.linalg.inv(L)

U_zero_rows = np.where(abs(np.diag(U)) < 1e-9)[0]
print(U_zero_rows)
L2 = L_inv[U_zero_rows,:]

[6 7]


In [23]:
theta =np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-0.22114233036131847-1.158297562487248e-15j],
       [ 0.2081603275712238 -4.887878443112358e-16j]])